### Checking for DAG-ness, running time

In [56]:
import numpy as np
import scipy.linalg as slin
import timeit

#### Generating a Matrix W

In [186]:
p, s = 100, 100

## Random matrix W with 50% edges
# W = np.random.randint(2, size = p ** 2).reshape(p, p)

## Random matrix W with s edges
# W = np.zeros(p ** 2)

# for i in range(s):
#     W[np.random.randint(p ** 2)] = 1
  
# W = W.reshape(p, p)

## Dense DAG
# W = np.zeros((p, p))
# W[np.tril_indices(p)] = 1

print(W)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 1. 1. ... 1. 1. 1.]]


#### Method 1.
Find row with all zeros, remove this row and column, repeat. This method is a bit slow as it also gives the corresponding ordering.

In [203]:
def is_dag(W_input):
    n = np.shape(W_input)[0]
    
    W = W_input.copy()
    # remove diagonal entries
    np.fill_diagonal(W, 0)
    
    order, old_order = [], list(range(n))
    
    # for the number of elements
    for i in range(n):
        
        # find a row that contains only zeros
        for j in range(n - i):
            # if we find a zero row (excl. diags)
            if not W[j].any() != 0:
                
                # remove this row and column
                W = np.delete(W, j, 0)
                W = np.delete(W, j, 1)
            
                order.append(old_order[j])
                old_order.remove(old_order[j])
                
                # go to next variable
                break
        
            # if no zero row exist stop
            elif i == n - 1:
                return False
            
    return True, order

is_dag(W)[0]

True

#### Method 2.
Find row with all zeros, remove this row and column, repeat. Same as method 2, but is a bit faster as it does not give the corresponding ordering.

In [204]:
def is_dag_2(W_input):
    
    p = np.shape(W_input)[0]
    
    W = W_input.copy()
    
    # remove diagonal entries
    np.fill_diagonal(W, 0)
    
    # for the number of elements
    for i in range(p):
        # find a row that contains only zeros
        for j in range(p - i):
            
            # if we find a zero row (excl. diags)
            if not W[j].any() != 0:
                
                # remove this row and column
                W = np.delete(W, j, 0)
                W = np.delete(W, j, 1)
                
                # go to next variable
                break
        
            # if no zero row exist stop
            elif i == p - 1:
                return False
            
    return True

is_dag_2(W)

True

#### Method 3.
Similar idea as method 2, but some faster computations with booleans and sums.

In [231]:
def is_dag_3(W_input):
    
    # create boolean copy, faster computations
    W = W_input.copy().astype(bool)
    
    # set diagonal to zero, as we do not care about self loops
    np.fill_diagonal(W, 0)
    
    # iteratively, for each variable
    for i in range(p):
        
        # compute the sum of rows and columns
        row_sums = W.sum(axis = 1)
    
        # check if there is a row with all zeros, aka no outgoing edges
        if np.min(row_sums) == 0:
            
            # get this row
            row = np.argmin(row_sums)
        
            # remove row and remove column
            W = np.delete(W, row, 0)
            W = np.delete(W, row, 1)
        
        # if we have not found such a row, we have no dag
        else: 
            return False
        
    # if we can remove all rows in such a way, we have a dag
    return True
    

is_dag_3(W)

False

#### Method 4.
NOTEARS criterion must be zero to be a DAG.

In [233]:
def h_notears(W):
    p = np.shape(np.array(W))[0]
    
    W2 = W.copy()

    np.fill_diagonal(W2, np.zeros(p))

    E = slin.expm(W2 * W2)  # (Zheng et al. 2018)
    h = np.trace(E) - p
    
    return h

def dag_notears(W):
    return h_notears(W) <= 0

dag_notears(W)

False

In [223]:
p, s, number = 100, 200, 100

W = np.zeros(p ** 2)

for i in range(s):
    W[np.random.randint(p ** 2)] = 1
  
W = W.reshape(p, p)

W[np.tril_indices(p, - 1)] = 0

print(f"Method 1 (SLOW w/ ORDER):\t{np.round(timeit.timeit('is_dag(W)', globals=globals(), number = number) / number * 1000, 3)} milliseconds.")
print(f"Method 2 (SLOW w/o ORDER):\t{np.round(timeit.timeit('is_dag_2(W)', globals=globals(), number = number) / number * 1000, 3)} milliseconds.")
print(f"Method 3 (FAST w/o ORDER):\t{np.round(timeit.timeit('is_dag_3(W)', globals=globals(), number = number) / number * 1000, 3)} milliseconds.")
print(f"Method 4 (NOTEARS):\t\t{np.round(timeit.timeit('dag_notears(W)', globals=globals(), number = number) / number * 1000, 3)} milliseconds.")

Method 1 (SLOW w/ ORDER):	20.971 milliseconds.
Method 2 (SLOW w/o ORDER):	20.912 milliseconds.
Method 3 (FAST w/o ORDER):	7.793 milliseconds.
Method 4 (NOTEARS):		6.953 milliseconds.


#### Verify Running time of Adjacency List to Adjacency Matrix

In [ ]:
def Lambda_to_adj(Lambda):
    """Convert Lambda list to adjacency matrix"""
    
    p = len(Lambda)
    
    adj_mat = np.zeros((p, p))
    
    for i, col in enumerate(Lambda):
        adj_mat[i, col] = 1 
    
    return adj_mat

In [92]:
# adj_list = [list(np.random.randint(p, size = 2)) for i in range(p)]
print(timeit.timeit('Lambda_to_adj(adj_list)', globals=globals(), number = 10000) / 10000) 

5.030325999999832e-05
